In [28]:
# Listing Datasets and cleaning the peep registry file
import pandas as pd

df = pd.read_csv("pp_peep_registry.csv")

peep_stats = df["peep_status"].isna()
df["peep_status"] = df["peep_status"].fillna(value="Not Available")
df["peep_required"] = df["peep_required"].astype(str).str.lower()

yes_values = ["y", "yes", "true"]
no_values = ["n", "no", "false"]

df["peep_required"] = df["peep_required"].apply(
    lambda x: "Yes" if x in yes_values else ("No" if x in no_values else "Not Specified")
)

mobility_category = df["mobility_category"].isna()
df["mobility_category"] = df["mobility_category"].fillna(value="Category Not Specified")

df["last_review_date"] = pd.to_datetime(df["last_review_date"], format="mixed", dayfirst=True, errors='coerce')

property_names = {"London HQ": "LDN-HQ", "Bristol Harbourside": "BRI-01", "Edinburgh Exchange" : "EDI-EX", "Manchester City Campus" : "MCH-CC", "Leeds Arena Point": "LDS-AR", "Birmingham Paradise": "BHM-PR", "Cardiff Bay": "CDF-BY", "Belfast Lagan": "BEL-LG", "Norwich Quarter": "NRW-QP", "Glasgow Clyde" : "GLW-CL"}
df["property_code"] = df["property_code"].replace(property_names)
df["property_code"] = df["property_code"].str.upper()
df = df[~((df["peep_required"] == "No") & (df["peep_status"] == "Not Available"))]
bad_statuses = ["Not Available", "Pending", "Expired", "Revoked"]
df["risk_status"] = df.apply(
    lambda row: "Potential" if (row["peep_status"] in bad_statuses and row["peep_required"] == "Yes") else "Not Potential",
    axis=1
)
df = df.drop_duplicates()
df.head(100)
df.to_csv("cleaned_peep_registry.csv", index=False)

In [ ]:
df2 = pd.read_csv("pp_property_occupancy_logs.csv")
df2["timestamp"] = pd.to_datetime(df2["timestamp"], format="%Y-%m-%d %H:%M:%S", errors="ignore", dayfirst=True, utc=True)
# df2["timestamp"] = pd.to_datetime(df2["timestamp"], errors="coerce", utc=True)
# df2["timestamp"] = df2["timestamp"]

df2["property_code"] = df2["property_code"].replace(property_names)
df2["property_code"] = df2["property_code"].str.upper()
df2["property_code"] = df2["property_code"].replace("Unknown", "")
df2["property_code"] = df2["property_code"].fillna("Not Available")
df2["property_code"] = df2["person_id"].map(df.set_index("person_id")["property_code"])

df2.loc[df2["property_code"].isna()]

df2["property_code"] = df2["property_code"].fillna(df["person_id"].map(
    df.set_index("person_id")["property_code"]
))
df2.head(35)

C:\Users\Alhaan Ahmed\AppData\Local\Temp\ipykernel_3340\1174500795.py:2: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  df2["timestamp"] = pd.to_datetime(df2["timestamp"], format="%Y-%m-%d %H:%M:%S", errors="ignore", dayfirst=True, utc=True)


AttributeError: Can only use .dt accessor with datetimelike values